In [5]:
import pandas as pd
import requests
import pymysql
import json
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from selenium import webdriver

In [32]:
host_name = "127.0.0.1"
username = "root"
password = "dlstjd88"
database_name = "steam_test"

db = pymysql.connect(
    host = host_name,
    port=3306,
    user=username,
    passwd=password,
    db=database_name,
    charset='utf8'
)

count = 570
chromedriver = '/home/nincastle/chromedriver/chromedriver'
tags_genre_query = []

def name_replace(name):
    name = name.replace("&","and").replace(".","dot")\
    .replace("Mod","Mod_").replace("'","").replace("-","_").replace("Linear","Linear_")\
    .replace("Action","Action_").replace("Foreign","Foreign_").replace("All","All_")
    return name

def driver_chrome(url, xpath_):
    driver_tags = webdriver.Chrome(chromedriver)
    driver_tags.get(url)
    print(type(xpath_))
    tags = driver_tags.find_element_by_xpath(xpath_)
    t_g = tags.find_elements_by_tag_name('a')
    driver_tags.quit
    return t_g

# 테그 변수 이름 크롤링
cro_tag = driver_chrome("https://steamdb.info/tags/","//div[@class='row list']")
for t in cro_tag:
    t = t.text.replace(" ","_")   
    tags_genre_query.append("tag_"+t+' INT(1)')


# 장르 변수 이름 크롤링
cro_gen = driver_chrome("https://steamdb.info/genres/","//div[@class='row']")
for g in cro_gen:
    g = g.text.replace(" ","_")
    tags_genre_query.append("genre_"+g+' INT(1)')

# driver_tags.quit
tag_genre_join = (", \n".join(tags_genre_query))

# SQL문 필터링
tag_genre_join = name_replace(tag_genre_join)
try: 
    cursor = db.cursor()
    SQL_QUERY = """
    DROP DATABASE IF EXISTS steam_test;
    CREATE DATABASE steam_test DEFAULT CHARSET=utf8 COLLATE=utf8_bin;

    USE steam_test;
    DROP TABLE IF EXISTS steam_test.steamtable;
    CREATE TABLE steam_test.steamtable (
        appid int(7),
        name varchar(100),
        owners int(10),
        owners_variance int(7),
        negative int(7),
        positive int(6),
        score_rank int(3),
        ccu int(6),
        players_2weeks int(8),
        players_forever_variance int(7),
        players_2weeks_variance int(7),
        userscore int(2),
        median_forever int(3),
        players_forever int(7),
        price int(7),
        average_2weeks int(5),
        developer varchar(50),
        median_2weeks int(4),
        publisher varchar(50),
        average_forever int(6),
        """+tag_genre_join+""",
        PRIMARY KEY(appid)
        ) DEFAULT CHARSET=utf8 COLLATE=utf8_bin
    """
    create_sql=("".join(SQL_QUERY))
#     print(create_sql)
    cursor.execute(create_sql)
    db.commit()
finally:
    print("table create ok!")
    pass

while count < 571:
    
    column_str=[]
    row_str=[]
#     not_tag_key=[]
    
    steamspy_url =('https://steamspy.com/api.php?request=appdetails&appid={}'.format(count))
    appdetails_response = requests.get(steamspy_url)
    data = appdetails_response.json()
#     print(appdetails_response.json())
    for data_k, data_v in data.items():
        data_k = name_replace(data_k).replace(" ","_")  
#         data_v = name_replace(data_v)
        if data_k == 'tags':
            for tag_k ,tag_v in data['tags'].items():
                tag_k = name_replace(tag_k).replace(" ","_")  
                column_str.append(("tag_{}").format(tag_k))
                row_str.append(1)
        else:
            column_str.append(data_k)
            row_str.append(data_v)
#             not_tag_key.append(data_k)
    print()
    try:
        cursor = db.cursor()
        sql = ("INSERT INTO steam_test.steamtable ("+(",".join(column_str)))
        sql = sql+(") VALUES {}".format(tuple(row_str)))
        print(sql)
        cursor.execute(sql)
        db.commit()
    finally:
        print("insert ok!")
        pass
                
    count += 10

<class 'str'>
<class 'str'>
table create ok!

INSERT INTO steam_test.steamtable (appid,median_forever,average_forever,tag_Simulation,tag_RPG,tag_Action__RPG,tag_Multiplayer,tag_Character_Customization,tag_Co_op,tag_Team_Based,tag_RTS,tag_Online_Co_Op,tag_PvP,tag_Replay_Value,tag_Competitive,tag_Free_to_Play,tag_Difficult,tag_MOBA,tag_Tower_Defense,tag_Action_,tag_Fantasy,tag_Strategy,tag_e_sports,median_2weeks,score_rank,negative,players_2weeks,developer,publisher,userscore,average_2weeks,owners,price,name,positive,players_forever,owners_variance,players_forever_variance,ccu,players_2weeks_variance) VALUES (570, 255, 11721, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 623, 64, 102501, 9403855, 'Valve', 'Valve', 87, 1152, 118517616, '0', 'Dota 2', 745900, 118517616, 280846, 280846, 795976, 89900)
insert ok!


INSERT INTO steam_test.steamtable (appid,median_forever,average_forever,tag_Simulation,tag_RPG,tag_Action__RPG,tag_Multiplayer,tag_Character_Customization,tag_Co_op,tag_Team_Based,tag_RTS,tag_Online_Co_Op,tag_PvP,tag_Replay_Value,tag_Competitive,tag_Free_to_Play,tag_Difficult,tag_MOBA,tag_Tower_Defense,tag_Action_,tag_Fantasy,tag_Strategy,tag_e_sports,median_2weeks,score_rank,negative,players_2weeks,developer,publisher,userscore,average_2weeks,owners,price,name,positive,players_forever,owners_variance,players_forever_variance,ccu,players_2weeks_variance) VALUES (570, 255, 11721, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 623, 64, 102501, 9403855, 'Valve', 'Valve', 87, 1152, 118517616, '0', 'Dota 2', 745900, 118517616, 280846, 280846, 795976, 89900)
